In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [5]:
!ls drive/cyclegan_proj/

combined_loss.py	    loss.py
data_sampler.py		    networks.py
inception_model_load.ipynb  __pycache__
inception_model.py	    train_colab_cyclegan.ipynb
inception_model.pyc	    train_pull.csv
inception_score.ipynb	    train_top.csv
inception_sham_model.pth    utl.py
inception_sham_state.pth


In [0]:
import sys
sys.path.append('drive/cyclegan_proj')

In [0]:
import torch
from inception_model import InceptionSham
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

In [0]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
def weights_init(m):
    classname = m.__class__.__name__
    if (classname.find('Conv')) != -1 and (classname.find('Basic') == -1):
        m.weight.data.normal_(0,
                              0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1,
                              0.02)
        m.bias.data.fill_(0)

In [0]:
model = torch.load("drive/cyclegan_proj/inception_sham_model.pth")

In [10]:
BATCH_SIZE = 30
N_EPOCH = 1
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE,
    shuffle=True)

Processing...
Done!


In [0]:
def test(model):
    model.eval()
    
    accuracy = []
    for i, (data, target) in enumerate(test_loader):
        data, target = Variable(data.cuda()), Variable(target.cuda())
        output = model(data).data.max(1)
        accuracy.append(torch.sum(model(data).data.max(1)[1] == target.data) / BATCH_SIZE)
        if i == 15:
            break
    return np.mean(accuracy)

In [18]:
test(model)

0.9958333333333333

In [0]:
inception_state = torch.load("drive/cyclegan_proj/inception_sham_state.pth")

In [0]:
inception = InceptionSham(num_classes=10, input_nc=1, dropout=0.5)
if torch.cuda.is_available():
  inception.cuda()
inception.load_state_dict(inception_state)

In [31]:
test(inception)

0.9979166666666667